## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [3]:
import pandas as pd

###  Importing Dataset

In [8]:
#Importing Dataset
df=pd.read_csv('C:\\Users\\91943\\PROJECT\\FakeNews Classifier LSTM\\fake-news\\train.csv')

In [9]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
##Drop Nan Values
df=df.dropna()

In [11]:
## Get the Independent Features
X=df.drop('label',axis=1)

In [12]:
## Get the Dependent features
y=df['label']

In [13]:
X.shape

(18285, 4)

In [14]:
y.shape

(18285,)

###  Data Preprocessing

In [15]:
import tensorflow as tf

In [16]:
tf.__version__

'2.7.0'

In [17]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [18]:
### Vocabulary size
voc_size=5000

In [19]:
messages=X.copy()

In [20]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [21]:
messages.reset_index(inplace=True)

In [22]:
import nltk
import re
from nltk.corpus import stopwords

In [23]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91943\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
print("Process Completed")

Process Completed


In [25]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [28]:
# One Hot Representation of Sentence
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[1]

[4164, 4503, 3815, 3097, 1673, 4182, 3287]

In [29]:
list_len = [len(i) for i in onehot_repr]
print(max(list_len))

47


### Embedding Representation

In [32]:
sent_length=47
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4558 3300 3827]
 [   0    0    0 ... 1673 4182 3287]
 [   0    0    0 ... 1250  562  858]
 ...
 [   0    0    0 ... 4989  414 3014]
 [   0    0    0 ... 1060 3540 2992]
 [   0    0    0 ... 3830  373 3349]]


In [33]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0, 1376, 3607, 4950, 4344, 3830, 1323,  853,
       4558, 3300, 3827])

In [34]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 47, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [35]:
len(embedded_docs),y.shape

(18285, (18285,))

In [36]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [37]:
X_final.shape,y_final.shape

((18285, 47), (18285,))

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=0)

### Model Training

In [40]:
# Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 16s 71ms/step - loss: 0.3365 - accuracy: 0.8363 - val_loss: 0.2126 - val_accuracy: 0.9135
Epoch 2/10
192/192 [==============================] - 13s 65ms/step - loss: 0.1465 - accuracy: 0.9421 - val_loss: 0.1900 - val_accuracy: 0.9229
Epoch 3/10
192/192 [==============================] - 13s 65ms/step - loss: 0.1026 - accuracy: 0.9636 - val_loss: 0.2088 - val_accuracy: 0.9208
Epoch 4/10
192/192 [==============================] - 13s 66ms/step - loss: 0.0783 - accuracy: 0.9736 - val_loss: 0.2342 - val_accuracy: 0.9167
Epoch 5/10
192/192 [==============================] - 13s 66ms/step - loss: 0.0550 - accuracy: 0.9834 - val_loss: 0.2313 - val_accuracy: 0.9152
Epoch 6/10
192/192 [==============================] - 13s 67ms/step - loss: 0.0403 - accuracy: 0.9881 - val_loss: 0.3053 - val_accuracy: 0.9152
Epoch 7/10
192/192 [==============================] - 13s 66ms/step - loss: 0.0266 - accuracy: 0.9929 - val_loss: 0.3520 - val_accuracy:

### Adding Dropout ( If Overfitting occurs )

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [47]:
y_pred=(model.predict(X_test) > 0.5).astype("int32")

In [48]:
y_pred

array([[1],
       [1],
       [1],
       ...,
       [0],
       [0],
       [1]])

In [49]:
from sklearn.metrics import confusion_matrix

In [50]:
confusion_matrix(y_test,y_pred)

array([[3098,  291],
       [ 253, 2393]], dtype=int64)

In [51]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9098591549295775